# Experiment 1 Setup

- Environemt is fixed with seed 4
- Map is undiscovered

General env_setup:
- Num rooms: 3
- Items: Souls (2), Health Potion (0-1), Enemies (Level 1, 1-2)

Aims:
The aim of this experiment is to check if the results from experiment 1 holds if we mainpulate the seed value.

In [2]:
!pip install stable-baselines3[extra] --quiet

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3 -m pip install --upgrade pip' command.


# Initialization Environment
Initialization and setup of the custom environment

In [2]:
from env import RoguesSoulsEnv

env = RoguesSoulsEnv(max_steps=500)
env.set_seed(4)
env.set_mode(1)
# Needs resetting to apply above changes
state = env.reset()
env.render(mode='human')

------------------------


                        
                        
                        
                        
     |------|           
     |....*.|           
     |......+####       
     |.>...b|   #       
     |..$...| |-+--|    
     |------| |.*..|    
              |..$b|    
              |....|    
              |....|    
    |----|    |....|    
    |....|    |-+--|    
    |....|      #       
    |.@..+#######       
    |....|              
    |....|              
    |----|              
                        
                        
                        
                        
		LEVEL: 1/1	  HP: 16 	SOULS: 0
		DEFENSE: 2	MAX HP: 16 	POWER: 4
------------------------


# Flattening the observation space

Stable-baseline3 requires the obseration space to be a 1D vector for better policy optimisation.


In [3]:
from gym.wrappers import FlattenObservation
env = FlattenObservation(env)

# Initialization agent
Agent will be based on stable-baselines3 DQN algorithm

In [4]:
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv

# Stop training when the model reaches the reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=140, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=callback_on_best, verbose=1)

env = DummyVecEnv([lambda: env])
model = DQN('MlpPolicy', env, verbose=1, exploration_fraction=0.3, tensorboard_log='./logs/ex2_0_v1')

Using cpu device


# Adding the necessary check function
This function checks the average score from n episodes

In [5]:
def get_average(env, model, n:int = 100) -> int:
    total = 0
    for e in range(1, n+1):
        obs = env.reset()
        done = False
        score = 0

        while not done:
            action, _ = model.predict(obs)
            obs, reward, done, _ = env.step(action)
            score += reward
        total += score

    return total/n

# Training the agent

The agent will be trained until its average performance is above 180.

In [6]:
# for i in range(5):
#     tot_steps = 25000
#     model.learn(total_timesteps = tot_steps, reset_num_timesteps=False)
#     avg = get_average(model, 25)
#     print(f'Total iterations: {(i+1)*tot_steps}; Average {avg}')
#     if avg > 100:
#         break
    
model.learn(total_timesteps = 800000, reset_num_timesteps=False, tb_log_name='first_run', callback=eval_callback)
avg = get_average(env, model, 25)
print(f'Average: {avg}')

Logging to ./logs/ex2_0_v1/first_run_0
----------------------------------
| rollout/            |          |
|    exploration rate | 0.992    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 189      |
|    time_elapsed     | 10       |
|    total timesteps  | 2000     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.984    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 241      |
|    time_elapsed     | 16       |
|    total timesteps  | 4000     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.976    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 261      |
|    time_elapsed     | 22       |
|    total timesteps  | 6000     |
----------------------------------
----------------

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=10000, episode_reward=-150.00 +/- 0.00
Episode length: 500.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 500      |
|    mean_reward      | -150     |
| rollout/            |          |
|    exploration rate | 0.96     |
| time/               |          |
|    total timesteps  | 10000    |
----------------------------------
New best mean reward!
----------------------------------
| rollout/            |          |
|    exploration rate | 0.96     |
| time/               |          |
|    episodes         | 20       |
|    fps              | 227      |
|    time_elapsed     | 43       |
|    total timesteps  | 10000    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.953    |
| time/               |          |
|    episodes         | 24       |
|    fps              | 238      |
|    time_elapsed     | 49       |
|    total ti

----------------------------------
| rollout/            |          |
|    exploration rate | 0.802    |
| time/               |          |
|    episodes         | 100      |
|    fps              | 247      |
|    time_elapsed     | 202      |
|    total timesteps  | 50000    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.794    |
| time/               |          |
|    episodes         | 104      |
|    fps              | 248      |
|    time_elapsed     | 209      |
|    total timesteps  | 52000    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00129  |
|    n_updates        | 499      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.788    |
| time/               |          |
|    episodes         | 108      |
|    fps              | 249      |
|    time_elapsed   

----------------------------------
| rollout/            |          |
|    exploration rate | 0.714    |
| time/               |          |
|    episodes         | 164      |
|    fps              | 244      |
|    time_elapsed     | 295      |
|    total timesteps  | 72200    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0257   |
|    n_updates        | 5549     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.707    |
| time/               |          |
|    episodes         | 168      |
|    fps              | 245      |
|    time_elapsed     | 301      |
|    total timesteps  | 73977    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00488  |
|    n_updates        | 5994     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rat

...

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 1904     |
|    fps              | 114      |
|    time_elapsed     | 5840     |
|    total timesteps  | 666041   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0436   |
|    n_updates        | 154010   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 1908     |
|    fps              | 114      |
|    time_elapsed     | 5845     |
|    total timesteps  | 667750   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.319    |
|    n_updates        | 154437   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rat

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 1964     |
|    fps              | 116      |
|    time_elapsed     | 5941     |
|    total timesteps  | 695000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0904   |
|    n_updates        | 161249   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 1968     |
|    fps              | 117      |
|    time_elapsed     | 5948     |
|    total timesteps  | 697000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.117    |
|    n_updates        | 161749   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rat

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 2024     |
|    fps              | 119      |
|    time_elapsed     | 6048     |
|    total timesteps  | 725000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.114    |
|    n_updates        | 168749   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 2028     |
|    fps              | 120      |
|    time_elapsed     | 6054     |
|    total timesteps  | 727000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0841   |
|    n_updates        | 169249   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rat

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 2084     |
|    fps              | 122      |
|    time_elapsed     | 6156     |
|    total timesteps  | 755000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.103    |
|    n_updates        | 176249   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 2088     |
|    fps              | 122      |
|    time_elapsed     | 6162     |
|    total timesteps  | 757000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0871   |
|    n_updates        | 176749   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rat

----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 2144     |
|    fps              | 125      |
|    time_elapsed     | 6261     |
|    total timesteps  | 783580   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.113    |
|    n_updates        | 183394   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 2148     |
|    fps              | 125      |
|    time_elapsed     | 6266     |
|    total timesteps  | 785127   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 5.93     |
|    n_updates        | 183781   |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rat

# Results

In [7]:
# import time
episodes = 20
print(f'TESTING MODEL ON {episodes} EPISODES\n')
for e in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
#         env.render()
#         input('[ENTER]')
#         time.sleep(1)
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    completed = info[0].get('Completed')
    print(f'Episode: {e}, Score: {score}, Completed: {completed}')
    

TESTING MODEL ON 20 EPISODES

Episode: 1, Score: [-34.99926], Completed: False
Episode: 2, Score: [-34.999268], Completed: False
Episode: 3, Score: [-34.99926], Completed: False
Episode: 4, Score: [-34.999268], Completed: False
Episode: 5, Score: [-34.99926], Completed: False
Episode: 6, Score: [-34.99926], Completed: False
Episode: 7, Score: [-84.99959], Completed: False
Episode: 8, Score: [-34.99926], Completed: False
Episode: 9, Score: [169.50067], Completed: True
Episode: 10, Score: [-34.99926], Completed: False
Episode: 11, Score: [-34.99926], Completed: False
Episode: 12, Score: [-34.99926], Completed: False
Episode: 13, Score: [-34.99926], Completed: False
Episode: 14, Score: [-34.99926], Completed: False
Episode: 15, Score: [-34.999268], Completed: False
Episode: 16, Score: [-34.99926], Completed: False
Episode: 17, Score: [-34.99926], Completed: False
Episode: 18, Score: [-34.99926], Completed: False
Episode: 19, Score: [-34.99926], Completed: False
Episode: 20, Score: [-34.99

In [8]:
## Continue training
# del model
# model = DQN.load('./models/experiment_1/ex7_1')
# model.learn(total_timesteps = 100000, reset_num_timesteps=False, tb_log_name='second_run')

In [9]:
# # Save model
# model.save('./models/experiment_1/ex12_v2')